In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import dotenv
from pathlib import Path

env_file = "../.env"

if os.path.exists(env_file):
    dotenv.load_dotenv(env_file, verbose=True)
    print("Loaded environment variables from .env file.")

cwd = os.getcwd()
# for some reason appending to PATH you need it to be string
sys.path.append(str(Path(cwd).parent / "src"))
hf_access_token = os.getenv("HUGGINGFACE_API_KEY")

# from research_tools.gpu import get_gpus_available

# os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(i) for i in get_gpus_available()])
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
# get rid of linker warnings that show up for some reason
os.environ["OTHER_LDFLAGS"] = "-w"

Loaded environment variables from .env file.


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List, Dict, Optional
import torch
from research_tools.utils import set_seed

set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
assert device == torch.device("cuda")

model_id = "LLM-LAT/zephyr7b-beta-rmu-lat-unlearn-wmdp-bio-cyber"
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id = "google/gemma-2b"
model_id = "meta-llama/Meta-Llama-3-8B"
model_id = "HuggingFaceH4/zephyr-7b-beta"

# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_id, torch_dtype=torch.bfloat16, trust_remote_code=True, token=hf_access_token
).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id

Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/mnt/align4_drive/data/huggingface/hub/models--HuggingFaceH4--zephyr-7b-beta/.no_exist/892b3d7a7b1cf10c7a701c60881cd93df615734c/adapter_config.json'
Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/mnt/align4_drive/data/huggingface/hub/models--HuggingFaceH4--zephyr-7b-beta/.no_exist/892b3d7a7b1cf10c7a701c60881cd93df615734c/adapter_config.json'


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [3]:
from unlearn_order.datasets.utils import DATASETS_DICT, Datasets


dataset_config = DATASETS_DICT[Datasets.WMDP_MCQ_CORPUS]

In [4]:
# ascent on corpus split, descent on wikitext
# eval loss on regular split mcq val split, wikitext val split
# also should eval on train split mcq val split, wikitext, mmlu cats
# all val files should be mcq
# all train should be corpus
# unless u do rtt

from datasets import load_dataset
from unlearn_order.dataset import load_dataset as load_dataset_unlearn

retain_dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

data_dir = Path("../data")

unlearn_files = dataset_config["unlearn_files"]
forget_train = load_dataset_unlearn(data_dir, unlearn_files)
forget_val = load_dataset_unlearn(data_dir, dataset_config["val_files"])

# retain_train = load_dataset("wikitext", "wikitext-2-raw-v1")["validation"]

retain_train = load_dataset_unlearn(data_dir, dataset_config["retain_files"])
retain_val = load_dataset_unlearn(data_dir, dataset_config["val_retain_files"])

val_files = dataset_config["val_files"]
n_val_files = 4


forget_train_1 = load_dataset_unlearn(data_dir, unlearn_files[:n_val_files])
forget_train_2 = load_dataset_unlearn(data_dir, unlearn_files[n_val_files:])
forget_val_1 = load_dataset_unlearn(data_dir, val_files[:n_val_files])
forget_val_2 = load_dataset_unlearn(data_dir, val_files[n_val_files:])

In [5]:
from unlearn_order.utils import (
    create_prompt_letter_answer,
    create_prompt,
    create_prompt_question_answer,
    create_prompt_answer_only,
)

completion_func = create_prompt_letter_answer

In [6]:
from datasets import Dataset
from transformers import AutoTokenizer
from functools import partial


def map_corpus(data: Dict, tokenizer: AutoTokenizer, max_length: int):
    text = data["text"]
    output = tokenizer(
        text,
        padding="max_length",
        max_length=max_length,
        truncation=True,
        return_tensors="pt",
    )
    return {
        "input_ids": output["input_ids"].squeeze(),
        "attention_mask": output["attention_mask"].squeeze(),
        "labels": output["input_ids"].clone().squeeze(),
    }


def process_train_dataset(
    dataset: Dataset,
    tokenizer: AutoTokenizer,
    max_length: int = 512,
    min_length: int = 0,
):
    dataset = dataset.filter(lambda x: len(x["text"]) > min_length)
    dataset = dataset.map(
        partial(map_corpus, tokenizer=tokenizer, max_length=max_length)
    )

    keep_cols = ["input_ids", "attention_mask", "labels"]
    dataset = dataset.remove_columns(
        [col for col in dataset.column_names if col not in keep_cols]
    )
    dataset.set_format("torch")
    return dataset


max_length = 512
forget_train = process_train_dataset(forget_train, tokenizer, max_length=max_length)
forget_train_1 = process_train_dataset(forget_train_1, tokenizer, max_length=max_length)
forget_train_2 = process_train_dataset(forget_train_2, tokenizer, max_length=max_length)
retain_train = process_train_dataset(retain_train, tokenizer, max_length=max_length)

Filter:   0%|          | 0/2355 [00:00<?, ? examples/s]

Map:   0%|          | 0/2355 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1884 [00:00<?, ? examples/s]

Map:   0%|          | 0/1884 [00:00<?, ? examples/s]

Filter:   0%|          | 0/471 [00:00<?, ? examples/s]

Map:   0%|          | 0/471 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [7]:
from copy import deepcopy
import torch.nn.functional as F


def create_mcq(
    record: Dict, tokenizer: AutoTokenizer, max_length: int, context: str = ""
):
    record["question"] = context + record["question"]
    text = completion_func(record)
    prompt = create_prompt(record)

    completion = text[len(prompt) :]

    record["prompt"] = prompt
    record["text"] = text
    record["completion"] = completion

    prompt_ids = tokenizer(prompt, return_tensors="pt").input_ids[0]
    completion_ids = tokenizer(
        completion, return_tensors="pt", add_special_tokens=False
    ).input_ids[0]

    input_ids = torch.cat([prompt_ids, completion_ids])
    labels = input_ids.clone()

    attention_mask = torch.ones_like(input_ids)
    prompt_mask = torch.zeros_like(input_ids)
    prompt_mask[: len(prompt_ids)] = 1

    completion_mask = torch.zeros_like(input_ids)
    completion_mask[len(prompt_ids) :] = 1

    # pad to max length on left
    seq_len = input_ids.size(0)
    pad_len = max_length - seq_len
    padding = (0, pad_len)

    input_ids = F.pad(input_ids, padding, value=0)
    labels = F.pad(labels, padding, value=-100)
    attention_mask = F.pad(attention_mask, padding, value=0)
    prompt_mask = F.pad(prompt_mask, padding, value=0)
    completion_mask = F.pad(completion_mask, padding, value=0)

    record["input_ids"] = input_ids
    record["labels"] = labels
    record["attention_mask"] = attention_mask
    record["completion_byte_len"] = len(completion.encode("utf-8"))
    record["prompt_mask"] = prompt_mask
    record["completion_mask"] = completion_mask
    record["length"] = seq_len

    return record


def expand_mcq_records(records: List[Dict], expand_choices: bool = True, **kwargs):
    new_records = []
    for rec in records:
        n_choices = len(rec["choices"])

        if not expand_choices:
            new_rec = deepcopy(rec)
            new_rec = create_mcq(new_rec, **kwargs)
            new_records.append(new_rec)
            continue

        for i in range(n_choices):
            new_rec = deepcopy(rec)
            actual_answer = rec["answer"]
            new_rec["answer"] = i
            new_rec = create_mcq(new_rec, **kwargs)
            new_rec["answer"] = actual_answer
            new_rec["selected_answer"] = i
            new_records.append(new_rec)

    return new_records

In [8]:
def expand_corpus_records(records: List[Dict]):
    for rec in records:
        rec["input_ids"] = torch.tensor(rec["input_ids"])
        rec["labels"] = torch.tensor(rec["labels"])
        rec["attention_mask"] = torch.tensor(rec["attention_mask"])
        rec["length"] = rec["input_ids"].size(0)
    return records

In [9]:
max_length = 1024

forget_train_records = forget_train.to_list()
retain_train_records = retain_train.to_list()
forget_val_records = forget_val.to_list()
retain_val_records = retain_val.to_list()

forget_train_records = expand_corpus_records(forget_train_records)
retain_train_records = expand_corpus_records(retain_train_records)
retain_val_records = expand_mcq_records(
    retain_val_records, tokenizer=tokenizer, max_length=max_length
)
forget_val_records = expand_mcq_records(
    forget_val_records, tokenizer=tokenizer, max_length=max_length
)

forget_train_records_1 = expand_corpus_records(
    forget_train_1.to_list(),
)
forget_train_records_2 = expand_corpus_records(
    forget_train_2.to_list(),
)

forget_val_1_train_records = expand_mcq_records(
    forget_val_1.to_list(), tokenizer=tokenizer, max_length=max_length, expand_choices=False
)
forget_val_2_train_records = expand_mcq_records(
    forget_val_2.to_list(), tokenizer=tokenizer, max_length=max_length, expand_choices=False
)

forget_val_1_records = expand_mcq_records(
    forget_val_1.to_list(), tokenizer=tokenizer, max_length=max_length
)
forget_val_2_records = expand_mcq_records(
    forget_val_2.to_list(), tokenizer=tokenizer, max_length=max_length
)

In [10]:
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm


def fix_seq_len(batch: Dict, keys: List[str]):
    max_seq_len = max(batch["length"])
    for key in keys:
        batch[key] = batch[key][:, :max_seq_len]
    return batch


def evaluate(
    model: AutoModelForCausalLM,
    records: List[Dict],
    batch_size: int = 8,
    n_choices: int = 4,
    normalize_loss: bool = False,
):
    # round up to nearest multiple of n_choices
    batch_size = (batch_size + n_choices - 1) // n_choices * n_choices
    original_fields = ["input_ids", "attention_mask", "labels"]

    aux_fields = ["answer", "completion_byte_len", "completion_mask", "length"]
    fields = original_fields + aux_fields
    dataset = [{k: v for k, v in rec.items() if k in fields} for rec in records]

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in tqdm(dataloader):
            batch = fix_seq_len(batch, original_fields + ["completion_mask"])
            input_ids = batch["input_ids"].to(device)
            labels = batch["labels"].to(device)
            completion_mask = batch["completion_mask"].to(device)
            attention_mask = batch["attention_mask"].to(device).bool()
            completion_byte_len = batch["completion_byte_len"].to(device)

            answers = batch["answer"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            logits = outputs.logits
            labels[~completion_mask.bool()] = -100
            labels[~attention_mask.bool()] = -100

            shifted_logits = logits[:, :-1, :].contiguous()
            shifted_labels = labels[:, 1:].contiguous()

            loss = F.cross_entropy(
                shifted_logits.view(-1, shifted_logits.size(-1)),
                shifted_labels.view(-1),
                reduction="none",
            )
            loss = loss.view(shifted_labels.size(0), shifted_labels.size(1))
            loss_by_sample = loss.sum(dim=1)

            if normalize_loss:
                loss_by_sample = loss_by_sample / completion_byte_len
            loss_by_sample = loss_by_sample.view(-1, n_choices)

            answers = answers[::n_choices]

            pred = loss_by_sample.argmin(dim=1)

            correct += (pred == answers).sum().item()
            total += len(answers)

    accuracy = correct / total
    return accuracy

In [11]:
from torch.utils.data import DataLoader
from transformers import Trainer, TrainingArguments
from unlearn_order.utils import log_1_minus_p_loss, my_loss
from unlearn_order.eval import eval_dataset as my_eval_dataset


def train_step_gd(
    model: AutoModelForCausalLM,
    forget_batch: Dict,
    retain_batch: Dict,
    forget_alpha: float = 0.1,
    use_log_1_minus_p: bool = True,
):
    forget_batch = fix_seq_len(forget_batch, ["input_ids", "attention_mask", "labels"])
    retain_batch = fix_seq_len(retain_batch, ["input_ids", "attention_mask", "labels"])

    forget_input_ids = forget_batch["input_ids"].to(device)
    forget_attention_mask = forget_batch["attention_mask"].to(device)
    forget_labels = forget_batch["labels"].to(device)

    retain_input_ids = retain_batch["input_ids"].to(device)
    retain_attention_mask = retain_batch["attention_mask"].to(device)
    retain_labels = retain_batch["labels"].to(device)

    forget_loss = my_loss(
        model,
        is_away=use_log_1_minus_p,
        input_ids=forget_input_ids,
        attention_mask=forget_attention_mask,
        labels=forget_labels,
    ) * (1 if use_log_1_minus_p else -1)

    forget_loss = forget_loss * forget_alpha
    forget_loss.backward()

    retain_loss = my_loss(
        model,
        is_away=False,
        input_ids=retain_input_ids,
        attention_mask=retain_attention_mask,
        labels=retain_labels,
    )

    retain_loss.backward()

    loss = forget_loss.detach() + retain_loss.detach()
    loss = loss.detach()
    forget_loss = forget_loss.detach()
    retain_loss = retain_loss.detach()

    loss_dict = {
        "forget_loss": forget_loss,
        "retain_loss": retain_loss,
        "loss": loss,
    }
    
    # check nans
    for k, v in loss_dict.items():
        if torch.isnan(v):
            raise ValueError(f"Loss {k} is NaN")

    return loss_dict

In [12]:
def train_epoch_gd(
    model: AutoModelForCausalLM,
    optimizer: torch.optim.Optimizer,
    epoch: int,
    forget_train_records: List[Dict],
    retain_train_records: List[Dict],
    batch_size: int = 4,
    forget_alpha: float = 0.1,
    log_steps: int = 50,
    grad_accum_steps: int = 1,
    use_log_1_minus_p: bool = True,
):
    model.train()
    forget_dataloader = DataLoader(
        forget_train_records, batch_size=batch_size, shuffle=True
    )
    retain_dataloader = DataLoader(
        retain_train_records, batch_size=batch_size, shuffle=True
    )
    loss_traj = []
    for step, (forget_batch, retain_batch) in tqdm(
        enumerate(zip(forget_dataloader, retain_dataloader))
    ):
        loss_dict = train_step_gd(
            model, forget_batch, retain_batch, forget_alpha, use_log_1_minus_p
        )

        if (step + 1) % grad_accum_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        loss_traj.append(loss_dict)
        if (step + 1) % log_steps == 0:
            print(
                f"Epoch {epoch}, Step {step}, Loss {loss_dict['loss']}, Forget Loss {loss_dict['forget_loss']}, Retain Loss {loss_dict['retain_loss']}"
            )

    return loss_traj


def train_gd(
    model: AutoModelForCausalLM,
    n_epochs: int,
    forget_train_records: List[Dict],
    retain_train_records: List[Dict],
    forget_val_records: List[Dict],
    retain_val_records: List[Dict],
    batch_size: int = 4,
    forget_alpha: float = 0.1,
    lr: float = 3e-5,
    log_steps: int = 50,
    eval_at_start: bool = True,
    grad_accum_steps: int = 1,
    use_log_1_minus_p: bool = False,
):
    if eval_at_start:
        retain_acc = evaluate(
            model, retain_val_records, batch_size=8, normalize_loss=False
        )
        forget_acc = evaluate(
            model, forget_val_records, batch_size=8, normalize_loss=False
        )

        print(f"Initial Retain Accuracy: {retain_acc}")
        print(f"Initial Forget Accuracy: {forget_acc}")

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(n_epochs):
        train_epoch_gd(
            model,
            optimizer,
            epoch,
            forget_train_records,
            retain_train_records,
            batch_size,
            forget_alpha,
            log_steps=log_steps,
            grad_accum_steps=grad_accum_steps,
            use_log_1_minus_p=use_log_1_minus_p,
        )
        retain_acc = evaluate(
            model, retain_val_records, batch_size=8, normalize_loss=False
        )
        forget_acc = evaluate(
            model, forget_val_records, batch_size=8, normalize_loss=False
        )

        print(f"Epoch: {epoch}, Retain Accuracy: {retain_acc}")
        print(f"Epoch: {epoch}, Forget Accuracy: {forget_acc}")
    return model

In [13]:
model = train_gd(
    model,
    5,
    forget_train_records_2,
    retain_train_records,
    forget_val_2_records,
    retain_val_records,
    4,
    forget_alpha=10,
    eval_at_start=False,
    log_steps=50,
    lr=1e-5,
    grad_accum_steps=8,
    use_log_1_minus_p=True,
)

50it [00:37,  1.33it/s]

Epoch 0, Step 49, Loss 2.627129077911377, Forget Loss 0.16817346215248108, Retain Loss 2.4589555263519287


100it [01:15,  1.35it/s]

Epoch 0, Step 99, Loss 1.988472580909729, Forget Loss 0.007305813021957874, Retain Loss 1.9811667203903198


118it [01:28,  1.34it/s]
100%|██████████| 79/79 [00:05<00:00, 14.96it/s]


...

Epoch 4, Step 49, Loss 1.646413803100586, Forget Loss 0.006644212640821934, Retain Loss 1.6397695541381836


100it [01:14,  1.35it/s]

Epoch 4, Step 99, Loss 1.5615534782409668, Forget Loss 0.0008613363606855273, Retain Loss 1.5606921911239624


118it [01:28,  1.34it/s]
100%|██████████| 79/79 [00:05<00:00, 14.98it/s]


Epoch: 4, Retain Accuracy: 0.5617834394904458
Epoch: 4, Forget Accuracy: 0.36942675159235666


In [14]:
val_1_acc = evaluate(model, forget_val_1_records, batch_size=8, normalize_loss=False)
val_2_acc = evaluate(model, forget_val_2_records, batch_size=8, normalize_loss=False)
print(f"Val 1 Accuracy: {val_1_acc}")
print(f"Val 2 Accuracy: {val_2_acc}")

100%|██████████| 79/79 [00:05<00:00, 14.93it/s]

Val 1 Accuracy: 0.445859872611465
Val 2 Accuracy: 0.36942675159235666


In [15]:
model = train_gd(
    model,
    3,
    forget_train_records_1,
    retain_train_records,
    forget_val_1_records,
    retain_val_records,
    4,
    forget_alpha=1,
    eval_at_start=False,
    log_steps=50,
    lr=1e-5,
    grad_accum_steps=8,
    use_log_1_minus_p=True,
)

50it [00:37,  1.32it/s]

Epoch 0, Step 49, Loss 1.964259147644043, Forget Loss 0.03389158099889755, Retain Loss 1.9303675889968872


100it [01:14,  1.35it/s]

Epoch 0, Step 99, Loss 1.6869665384292603, Forget Loss 0.04892630875110626, Retain Loss 1.6380401849746704


150it [01:52,  1.35it/s]

...

Epoch 2, Step 399, Loss 0.9962016940116882, Forget Loss 0.00257524149492383, Retain Loss 0.9936264753341675


450it [05:36,  1.33it/s]

Epoch 2, Step 449, Loss 1.0581632852554321, Forget Loss 0.002496466040611267, Retain Loss 1.0556668043136597


471it [05:52,  1.34it/s]
100%|██████████| 314/314 [00:20<00:00, 15.53it/s]

Epoch: 2, Retain Accuracy: 0.575796178343949
Epoch: 2, Forget Accuracy: 0.35509554140127386


In [16]:
val_1_acc = evaluate(model, forget_val_1_records, batch_size=8, normalize_loss=False)
val_2_acc = evaluate(model, forget_val_2_records, batch_size=8, normalize_loss=False)
print(f"Val 1 Accuracy: {val_1_acc}")
print(f"Val 2 Accuracy: {val_2_acc}")

100%|██████████| 79/79 [00:05<00:00, 14.97it/s]

Val 1 Accuracy: 0.35509554140127386
Val 2 Accuracy: 0.3375796178343949


In [17]:
from torch.utils.data import DataLoader
from transformers import Trainer, TrainingArguments
from unlearn_order.utils import log_1_minus_p_loss, my_loss


def train_step_ft(
    model: AutoModelForCausalLM,
    batch: Dict,
):
    batch = fix_seq_len(
        batch, ["input_ids", "attention_mask", "labels", "completion_mask"]
    )

    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)
    completion_mask = batch["completion_mask"].to(device)

    # only train on completions for multiple choice
    labels[~completion_mask.bool()] = -100

    output = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
    loss = output.loss

    loss.backward()

    loss = loss.detach().item()

    loss_dict = {
        "loss": loss,
    }

    return loss_dict

In [18]:
# save model to ../models/ directory
model_dir = Path("../models/zephyr_AB")
model.save_pretrained(model_dir)
# load model from checkpoint
model = AutoModelForCausalLM.from_pretrained(model_dir, torch_dtype=torch.bfloat16).to(
    device
)
# model_dir.mkdir(exist_ok=True)

# save checkpoint
# model.save_pretrained(model_dir)

[2024-11-22 17:27:56,438] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/mnt/align1_drive/tcqian/miniconda/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/mnt/align1_drive/tcqian/miniconda/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/mnt/align1_drive/tcqian/miniconda/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/mnt/align1_drive/tcqian/miniconda/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/mnt/align1_drive/tcqian/miniconda/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/mnt/align1_drive/tcqian/miniconda/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/mnt/align1_dri

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [19]:
def train_epoch_ft(
    model: AutoModelForCausalLM,
    optimizer: torch.optim.Optimizer,
    epoch: int,
    mcq_records: List[Dict],
    batch_size: int = 4,
    lr: float = 3e-5,
    log_steps: int = 50,
    grad_accum_steps: int = 1,
):
    model.train()
    dataloader = DataLoader(mcq_records, batch_size=batch_size, shuffle=True)

    loss_traj = []
    for step, batch in tqdm(enumerate(dataloader)):
        loss_dict = train_step_ft(model, batch)

        if (step + 1) % grad_accum_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        loss_traj.append(loss_dict)
        if (step + 1) % log_steps == 0:
            print(f"Epoch {epoch}, Step {step}, Loss {loss_dict['loss']}")

    return loss_traj


def train_ft(
    model: AutoModelForCausalLM,
    n_epochs: int,
    mcq_records: List[Dict],
    forget_val_1_records: List[Dict],
    forget_val_2_records: List[Dict],
    batch_size: int = 4,
    lr: float = 3e-5,
    log_steps: int = 50,
    eval_at_start: bool = True,
    grad_accum_steps: int = 1,
):
    if eval_at_start:
        forget_acc_1 = evaluate(
            model, forget_val_1_records, batch_size=8, normalize_loss=False
        )
        forget_acc_2 = evaluate(
            model, forget_val_2_records, batch_size=8, normalize_loss=False
        )

        print(f"Initial Forget accuracy 1: {forget_acc_1}")
        print(f"Initial Forget accuracy 2: {forget_acc_2}")

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(n_epochs):
        train_epoch_ft(
            model,
            optimizer,
            epoch,
            mcq_records,
            batch_size,
            lr=lr,
            log_steps=log_steps,
            grad_accum_steps=grad_accum_steps,
        )
        forget_acc_1 = evaluate(
            model, forget_val_1_records, batch_size=8, normalize_loss=False
        )
        forget_acc_2 = evaluate(
            model, forget_val_2_records, batch_size=8, normalize_loss=False
        )

        print(f"Epoch {epoch}, Forget accuracy 1: {forget_acc_1}")
        print(f"Epoch {epoch}, Forget accuracy 2: {forget_acc_2}")

    return model

In [20]:
train_ft(
    model,
    10,
    forget_val_1_train_records,
    forget_val_1_records,
    forget_val_2_records,
    batch_size=4,
    lr=1e-6,
    log_steps=50,
    eval_at_start=False,
    grad_accum_steps=1,
)

50it [00:14,  3.48it/s]

Epoch 0, Step 49, Loss 0.2544366717338562


100it [00:28,  3.64it/s]

Epoch 0, Step 99, Loss 0.07201399654150009


150it [00:43,  3.44it/s]

...

150it [00:43,  3.62it/s]

Epoch 9, Step 149, Loss 0.00021163173369131982


157it [00:45,  3.45it/s]
100%|██████████| 79/79 [00:05<00:00, 15.00it/s]

Epoch 9, Forget accuracy 1: 1.0
Epoch 9, Forget accuracy 2: 0.5222929936305732


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), e